

```
# BERT - Feature Extraction
```



In [ ]:
# receives
# .txt file containing a list of question-answer pairs identified by 'P: ' and 'R: '
# .txt file with a list of questions, one question per line

# retrieves
# .txt file containing the posed questions and respective BERT's answers, identified by 'P: ' and 'R: '

In [ ]:
pip install transformers

In [2]:
import transformers
from transformers import pipeline
import numpy
import scipy

In [3]:
# file management
def open_file(filename):
    read_file = open(filename, 'r')
    file_cont = read_file.readlines()
    read_file.close()

    return file_cont

def write_file(filename, content):
    file_write = open(filename, 'w')
    file_write.writelines(content)
    file_write.close()

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# BERT - feature extraction pipeline
def bert_model_fe(model_name, pipeline_name):
    pipe_feat_extraction = pipeline(pipeline_name, model=model_name)
    return pipe_feat_extraction

def get_vector(question, pipe_feat_extraction):
    vector = pipe_feat_extraction(question)
    result = vector[0][0]

    return result

pipe_feat_extraction = bert_model_fe('neuralmind/bert-large-portuguese-cased', "feature-extraction")
print('Feature Extraction Model Downloaded')

In [11]:
# returns the answer corresponding to the question in position question_index of question-answer pairs file
def get_answer(file_content, question_index):
    answer = ''

    for i in range(question_index + 1, len(file_content)):
        if file_content[i] == '\n' or 'P: ' in file_content[i]:
            break
        else:
            answer = answer + file_content[i]

    return answer

In [7]:
# computes all questions embeddings, and stores them along with the question position in the domain file
def domain_questions_embeddings(domain_content, pipe_feat_extraction):
    all_emb = []

    for i in range(len(domain_content)):
        emb_pos = []

        if 'P: ' in domain_content[i]:
            aux_ques = domain_content[i].replace('P: ', '')
            aux_emb = get_vector(aux_ques, pipe_feat_extraction)

            emb_pos.append(aux_emb)
            emb_pos.append(i)

            all_emb.append(emb_pos)

    return all_emb

In [8]:
# returns position of question in domain most similar to posed question
def compute_similarity(all_emb, ques_emb):
    cos_sim = -1
    pos = 0

    for i in range(len(all_emb)):
        aux_cos_sim = 1 - scipy.spatial.distance.cosine(all_emb[i][0], ques_emb)

        if aux_cos_sim > cos_sim:
            cos_sim = aux_cos_sim
            pos = all_emb[i][1]

    return pos

In [13]:
# retrieves a file with the posed questions and BERT's answers
def bert_fe(questions_file_path, domain_file_path, pipe_feat_extraction, save_file_path):
    final_file_content = []
    questions = open_file(questions_file_path)
    domain_content = open_file(domain_file_path)

    # creates BERT vector representations of all questions in the domain file
    all_emb = domain_questions_embeddings(domain_content, pipe_feat_extraction)

    for i in range(len(questions)):

        # creates BERT vector representations of posed question
        ques_emb = get_vector(questions[i], pipe_feat_extraction)

        # returns position of question in domain most similar to posed question
        best_match_pos = compute_similarity(all_emb, ques_emb)

        # creates the final file containing all posed questions and respective retrieved answers, with 'P: ' and 'R: ' identifiers
        aux_ques = 'P: ' + questions[i]
        aux_ans = get_answer(domain_content, best_match_pos)

        final_file_content.append(aux_ques)
        final_file_content.append(aux_ans)
        final_file_content.append('\n')

    write_file(save_file_path, final_file_content)
    print('File with posed questions and respective answers created!')

In [ ]:
# NOTES

# domain_file_path - path to the file containing the domain
# must be a file containing question-answer pairs identified with 'P: ' and 'R: ', respectively
# FAQs                      
# P: question1             
# R: answer1           
# \n                        
# P: question2              
# R: answer2                
# \n                       
# must be a .txt file

# questions_file_path - path to the file containing all questions, one question per line
# Q1
# Q2
# Q3
# ...
# must be a .txt file

# save_file_path - path to the file where the posed questions and respective retrieved answers are to be saved
# must be a .txt file

In [ ]:
domain_file_path = # 'domain_file_path'
questions_file_path = # 'questions_file_path'
save_file_path = # 'save_file_path'

bert_fe(questions_file_path, domain_file_path, pipe_feat_extraction, save_file_path)

In [14]:
domain_file_path = 'AlticeFAQs.txt'
questions_file_path = 'Test_AlticeFAQs.txt'
save_file_path = 'Results.txt'

bert_fe(questions_file_path, domain_file_path, pipe_feat_extraction, save_file_path)